In [128]:
'''
make program that automatically slice sequence to the length desired
the output filenames will have the following:
1. shortname
2. chromosome name
3. start, stop, position, index start with 1(instead of 0)
'''
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
META_SEQ_LENGTH = 10000

import subprocess
import os
import re

In [74]:
chromosome = SeqIO.parse("../5_Tests/Sep26SlicingGenome/Schpo1_chrome1.fasta",'fasta').next()
print(chromosome.id)
print(chromosome.seq[:1000])
print(len(chrome1))

chromosome_1
GATCACGTACATCACCTTGTAAGAATTTATCTGCAATAGTCCTTCGGTATTGTACATTGTTCCAAGCATAGTAAACTAACGATATCAAGTTTGCCTTTCTAGCCCATGACCTACAGTCAGAAGTGTAAGCCATATCACTGTCGGCATGTTCAAACTTTGTCAAACCACAAAATAAACAAAGTCCTTGAAATCGAATACGTAGTTTACATTCTCGCAAGTTGTGGTCGGCCTTGCCACATTTATAACAAGTAGATAAGCGTACGGGGCATGCTTTCCCGGTATGAGCACGAATTTCTGTGTCTGGGTTACCAAGAGTGCAACTTAGACATTCATCTTTATACACTCGAAAGAATCGCTCGAGTTGTGCTGAAATGTCAGTTGAGTCTACCCATTGTTTTTTGACGGCTTGCACACGACTTTTATACTCGGCGAAATGAATGGTACGAGAATCAACATCACCATCACCATCACCATCACCAATACCAATACCAATACCAACAATACCACTCATACCTCCAACACTACCATGAACACGATTCATGTCATGGTTGACGCCCTGTTGTACATTGTTCAAATGTTCATCGTATGTGCGTTTATGACCACGACGATGGCTGAGCGACATGTCTTCCAAACCATACTTTAGATTCGTATTAGCATTGCCAAAGCTTTTCTTCCATGTTGGCGATACAGACGAAGGGAACAAAGTGATTTCCGATAATTCCGACAAAGACACTGGCGTAGTACTAGTAGTGTTAGTGTTCATAGCACTCATACCACTGCTATTCCCTGGCTGTGGTGGGGGGGAAGACGAAAACGATGCATTATAACGCGAATGTGTTGCAATGGCTGGTTTCGGTGTTTCTGGCAATGTCGGTTTCACTCCATACATCGTAGACACAGTTGATTCTTCACCAAGTAACGAATCTGAGCATCTTGAGCAGTAAACACAGTTAGCAAACGATGCACAACATACACATTCGCCATCCA

In [21]:
chrome1.seq[:1000000000000]

Seq('GATCACGTACATCACCTTGTAAGAATTTATCTGCAATAGTCCTTCGGTATTGTA...ATC', SingleLetterAlphabet())

In [22]:
meta_seqs = [chrom1.seq[i:i+META_SEQ_LENGTH] for i in range(0,len(chrom1),META_SEQ_LENGTH)]

In [44]:
len(meta_seqs)

558

In [37]:
shortName = 'Schpo1'
piece = 'chromosome_1'
position = '1to10000'

In [41]:
test1 = SeqRecord(meta_seqs[0],id = '{}|{}|{}'.format(shortName,piece,position), description="")

In [42]:
test1

SeqRecord(seq=Seq('GATCACGTACATCACCTTGTAAGAATTTATCTGCAATAGTCCTTCGGTATTGTA...CGG', SingleLetterAlphabet()), id='Schpo1|chromosome_1|1to10000', name='<unknown name>', description='', dbxrefs=[])

In [43]:
SeqIO.write(test1,'../5_Tests/Sep26SlicingGenome/{}_{}_{}.fasta'.format(shortName,piece,position), 'fasta')

1

In [105]:
def sliceToPieces(filename, outputPath, shortName, interval = 10000, piece_needed = 50, is_piece_limited = True):
    '''
    slice chromosome into the pieces with the length required 
    applicable to only one chromosome per file
    '''
    chromosome = SeqIO.parse(filename,'fasta').next()
    meta_seqs = [((i,i+interval),chromosome.seq[i:i+interval]) for i in range(0,len(chromosome),interval)]
    counter = 0
    for pair in meta_seqs:
        if is_piece_limited and counter == piece_needed:
            break
        counter+=1
        location, meta_piece = pair
#         print location
        position = "{}_{}".format(int(location[0])+1,int(location[1]))
#         print position
        sub_seq = SeqRecord(meta_piece,id = '{}|{}|{}'.format(shortName,chromosome.id,position), description="")
        SeqIO.write(sub_seq,'{}{}_{}_#{}_{}.fasta'.format(outputPath,shortName,chromosome.id,counter,position), 'fasta')

In [107]:
filename = '../5_Tests/Sep26SlicingGenome/Schpo1_chrome1.fasta'
outputPath = '../5_Tests/Sep26SlicingGenome/SlicedSchpo1/'
shortName = "Schpo1"
sliceToPieces(filename, outputPath, shortName, is_piece_limited=False)

In [124]:
## generate scripts that run genemark.hmm with given model file
# modelFilePath = '/home/richard/research/5_Tests/Sep5modelWithLengthDistribution/modelDIY/'
# modelFileName = 'GC36IntronSpacerDonorToBPSingleInternalInitialTerminal.txt'

modelFilePath = '/home/richard/research/5_Tests/Sep5modelWithLengthDistribution/'
modelFileName = 'Schpo1.ES_C_4.mod'

fastaFilePath = '/home/richard/research/5_Tests/Sep26SlicingGenome/SlicedSchpo1/'
fastaFileNames = filter(lambda x: 'fasta' in x, os.listdir(fastaFilePath))
outputPath = '/home/richard/research/5_Tests/Sep26SlicingGenome/es_result_Schpo1/'


## need to run under node001 to node036
def genemarkHMM_script(modelFilePath,modelFileName,fastaFilePath,fastaFileNames,outputPath):
    script = ''
    base = "/scratch/bin/essuite/gmhmme3 "
    model = '-m {}{} '.format(modelFilePath,modelFileName)
    for fastaFile in fastaFileNames:   
        fasta = "{}{} ".format(fastaFilePath,fastaFile)
        output = "-o {}{}{}.es_output".format(outputPath,fastaFile[:-5],modelFileName[:-4])
        script += base + model + fasta + output+"\n"
#     print script
#     subprocess.call(script,shell = True)
#     print script
    return script
# genemarkHMM_script(modelFilePath,modelFileName,fastaFilePath,fastaFileNames,outputPath)
script = genemarkHMM_script(modelFilePath,modelFileName,fastaFilePath,fastaFileNames,outputPath)
# print script
with open("/home/richard/research/5_Tests/Sep26SlicingGenome/ESFullModMetaScript.sh",'w') as f: f.write(script)

In [221]:
## assemble prediction
predictionPath = '/home/richard/research/5_Tests/Sep26SlicingGenome/es_result_Schpo1/'
predictionNames = filter(lambda x: 'es_output' in x, os.listdir(predictionPath))

pattern = '\#(\d*)\_' #main pattern to get rid of over head
pattern_interval = '\#\d*_(\d*)_(\d*)\.'
substitute_pattern = "\d+?"
prediction_pattern = '(\d+?)\s+?\d\s+?[+|-]\s+?[^-+]'

# a,b = re.findall(pattern_interval,predictionNames[0])[0]
# print int(b)-int(a)+1

def find_idx(predictionName): return int(re.findall(pattern,predictionName)[0]) # find index of the output file
def find_gene_number(line): 
    try:
        return int(re.findall(substitute_pattern,line)[0])
    except:
        print "problem with line: ", line
def substitute_gene_number(line, number, gene_count): 
#     print number+gene_count
#     print line
    return  re.sub(substitute_pattern,str(number+gene_count),line, count = 1)
def substitute_exon_range(line, i, interval): 
    line_elem = line.split()
    line_elem[4], line_elem[5] = str(int(line_elem[4])+interval*i), str(int(line_elem[5])+interval*i)
    return ' '.join(line_elem)

def assemble_annotation(predictionPath, predictionNames):
    idx_file_pair = {find_idx(predictionName):predictionName for predictionName in predictionNames}
    
    ##locate seperate line and overhead
    with open(predictionPath+predictionNames[0]) as f:
        sep_line = ''
        with open(predictionPath+predictionNames[0]) as f1:
            for line in f1:
                if '#' in line and 'Type' in line:
                    sep_line = line     #sep
                    break
        text = f.read()
        parts = text.split(sep_line)
        base = parts[0] + sep_line      #overhead for annotation

        gene_counter = 0 # for all the predictions

        for idx in idx_file_pair:
            number = find_idx(idx_file_pair[idx]) #locate the rank number of the prediction file 
            a,b = re.findall(pattern_interval,predictionNames[0])[0]
            interval = int(b)-int(a)+1
            
#             print idx_file_pair[idx]
            with open(predictionPath+idx_file_pair[idx]) as f:
                all_line = filter(lambda x: len(x) > 10,f.read().split(sep_line)[1].split('\n')) #get rid of empty line
#                 print all_line
                if len(all_line) == 0: break #it's possible that there is no gene in a file, so no prediction
        
                
                gene_numbers = map(find_gene_number, all_line) # find gene number for each line
                
                #first modi
                modified_all_line = [substitute_gene_number(all_line[i], gene_numbers[i], gene_counter) \
                                     for i in range(len(all_line))]
                #idx-1 is the number of interval so far
                modified_all_line = [substitute_exon_range(modified_all_line[i], int(idx)-1, interval) 
                                     for i in range(len(all_line))]
                modified_all_line += '\n'
                gene_counter += len(set(gene_numbers))
            base+='\n'.join(modified_all_line)
    return base

# print find_gene_number('  2   1   -  Single            5726       6331        606          3 1 - -')
# print int(re.findall(substitute_pattern,'  2   1   -  Single            5726       6331        606          3 1 - -')[0])
# substitute_gene_number('  2   1   -  Single            5726       6331        606          3 1 - -', 20, 0)
res = assemble_annotation(predictionPath, predictionNames)  
print res[:2000]

Eukariotyc GeneMark.hmm version 3.48
Sequence name: /home/richard/research/5_Tests/Sep26SlicingGenome/SlicedSchpo1/Schpo1_chromosome_1_#87_860001_870000.fasta
FASTA defline: >Schpo1|chromosome_1|860001_870000
Sequence length: 10000 bp
G+C content: 35.71%
Matrices file: /home/richard/research/5_Tests/Sep5modelWithLengthDistribution/Schpo1.ES_C_4.mod
Sun Sep 27 00:52:18 2015


Predicted genes/exons

Gene Exon Strand Exon           Exon Range         Exon      Start/End
  #    #         Type                             Length       Frame
1 1 - Initial 11 5662 5652 3 1 - -
2 1 - Single 5726 6331 606 3 1 - -
3 1 + Initial 7579 7749 171 1 3 - -
3 2 + Internal 7791 8002 212 1 2 - -
3 3 + Terminal 8053 9274 1222 3 3 - -

4 1 + Single 12158 12994 837 1 3 - -
5 1 + Single 15855 16226 372 1 3 - -
6 1 + Initial 17469 17572 104 1 2 - -
6 2 + Terminal 17686 18043 358 3 3 - -
7 1 + Single 18117 18557 441 1 3 - -

8 1 + Initial 20827 20932 106 1 1 - -
8 2 + Internal 21552 22076 525 2 1 - -
8 3 + Termi

In [215]:
with open("../5_Tests/Sep27AssembleAnnotation/ESFullAssembledAnnotation.es_output",'w') as f: f.write(res)

In [ ]:
## compare with annotation
## filter out the annotation that is cut


In [ ]:
##################### testing ##################################

In [212]:
string.split()

['1', '1', '-', 'Initial', '11', '5662', '5652', '3', '1', '-', '-']

In [198]:
# prediction_pattern = '(\d+?)\s+?\d\s+?[+|-]\s+?[^-+]'
substitute_pattern = "\d+?"
string = """  1   1   -  Initial             11       5662       5652          3 1 - -"""

re.findall(prediction_pattern,string)

#replace the gene count in an accumulated fashion
# re.sub(substitute_pattern,'abc',string, count = 1)


['1']